In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import torch
import wandb

from denoising import Environment, prepare_training
from denoising.configs import TrainConfig
from denoising.utils import seed_everything

In [3]:
CWD = Path.cwd()

In [4]:
env = Environment(_env_file=CWD / '../env')

In [5]:
wandb.login(key=env.wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/d.nesterov/.netrc
wandb: Currently logged in as: dmitrylala to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Конфиг обучения и подготовка

In [6]:
run_idx = 17
save_dir = CWD / 'mri'

cfg = TrainConfig(
    # Datasets params
    train_dset='mri_pm_train',
    test_dset='mri_pm_test',
    train_batch_size=128,
    test_batch_size=256,

    # Model params
    name_model='mri-hno-v2-retrain',
    cfg_fno={
        'n_modes': (32, 32),
        'in_channels': 1,
        'hidden_channels': 32,
        'lifting_channel_ratio': 8,
        'projection_channel_ratio': 2,
        'out_channels': 1,
        'factorization': 'dense',
        'n_layers': 4,
        'rank': 0.42,
        'spectral': 'hartley',
    },

    # Run params
    random_seed=42,
    device='cuda:2',
    run_name=f'Run {run_idx}, HNO v2 (retrain)',
    save_weights_path=save_dir / f'run-{run_idx}-weights.pt',

    # Train params
    n_epochs=50,
    lr=1e-3,
    wandb_log=False,
    save_dir=save_dir / f'run-{run_idx}',
    verbose=True,
)

In [7]:
trainer, train_kwargs, run = prepare_training(env, cfg)

Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_test           with sample size = torch.Size([3, 512, 512])
torch.Size([128, 1, 145, 145]) torch.Size([128, 1, 145, 145])
Loaded  model mri-fno-neuralop with n_parameters = 2010449
Loaded  model mri-fno-custom   with n_parameters = 2010449
Loaded  model mri-fno-dense    with n_parameters = 4476513
Loaded  model mri-hno-separable with n_parameters = 2248289
Loaded  model 

# Запуск

In [19]:
seed_everything(cfg.random_seed)
trainer.train(**train_kwargs)

In [16]:
if run is not None:
    run.finish()

In [20]:
torch.save(trainer.model.to('cpu').state_dict(), cfg.save_weights_path)